In [10]:
from datasets import load_dataset
single_ds = load_dataset("NousResearch/hermes-function-calling-v1", "func_calling_singleturn")
multi_ds = load_dataset("NousResearch/hermes-function-calling-v1", "func_calling")
glaive_ds = load_dataset("NousResearch/hermes-function-calling-v1", "glaive_func_calling")

In [100]:
import json
import re
from pprint import pprint


def map_func(example):
    messages = []
    if example["id"] == "e39781a4-29a3-4896-83d2-0ab5d264b6ed":
        example["conversations"] = messages
        example["functions"] = "[]"
        return example
    try:
        for c in example["conversations"]:
            if c["from"] == "system":
                tools = re.search(r'<tools>\n(.*?)\n</tools>', c["value"], re.DOTALL).group(1)
                example["functions"] = json.dumps(eval(tools), ensure_ascii=False)
            elif c["from"] == "human":
                messages.append({"role": "user", "content": c["value"]})
            elif c["from"] == "gpt":
                tool_calls = re.findall(r'<tool_call>\n(.*?)\n</tool_call>', c["value"], re.DOTALL)
                if tool_calls:
                    tool_calls = json.dumps([eval(t) for t in tool_calls], ensure_ascii=False)
                    content = re.search(r'^(.*?)<tool_call>\n', c["value"], re.DOTALL).group(1)
                else:
                    content = c["value"]
                    tool_calls = "[]"
                messages.append({"role": "assistant", "content": content, "function_call": tool_calls})
            elif c["from"] == "tool":
                tool_rsps = re.findall(r'<tool_response>\n(.*?)\n</tool_response>', c["value"], re.DOTALL)
                tool_rsps = json.dumps([eval(t) for t in tool_rsps], ensure_ascii=False)
                messages.append({"role": "function", "content": tool_rsps})
    except:
        messages = []
        example["functions"] = "[]"
    example["conversations"] = messages
    return example


glaive_ds["train"].map(map_func).filter(lambda x: x["conversations"])

Filter: 100%|██████████| 5209/5209 [00:00<00:00, 89076.06 examples/s]


Dataset({
    features: ['id', 'conversations', 'tools', 'category', 'subcategory', 'task', 'source', 'functions'],
    num_rows: 0
})

In [108]:
def glaive_map_func(example):
    messages = []
    try:
        for c in example["conversations"]:
            if c["from"] == "human":
                messages.append({"role": "user", "content": c["value"]})
            elif c["from"] == "gpt":
                tool_calls = re.findall(r'<tool_call>\n(.*?)\n</tool_call>', c["value"], re.DOTALL)
                if tool_calls:
                    tool_calls = json.dumps([eval(t) for t in tool_calls], ensure_ascii=False)
                    content = re.search(r'^(.*?)<tool_call>\n', c["value"], re.DOTALL).group(1)
                else:
                    content = c["value"]
                    tool_calls = "[]"
                messages.append({"role": "assistant", "content": content, "function_call": tool_calls})
            elif c["from"] == "tool":
                tool_rsps = re.findall(r'<tool_response>\n(.*?)\n</tool_response>', c["value"], re.DOTALL)
                tool_rsps = json.dumps([eval(t) for t in tool_rsps], ensure_ascii=False)
                messages.append({"role": "function", "content": tool_rsps})
    except:
        messages = []
    if json.loads(example["tools"]):
        try:
            example["functions"] = json.dumps([t["function"] for t in json.loads(example["tools"])], ensure_ascii=False)
        except:
            messages = []
            example["functions"] = "[]"
    else:
        example["functions"] = "[]"
    example["conversations"] = messages
    return example


glaive_ds["train"].map(glaive_map_func).filter(lambda x: x["conversations"])

Filter: 100%|██████████| 5209/5209 [00:00<00:00, 29842.97 examples/s]


Dataset({
    features: ['id', 'conversations', 'tools', 'category', 'subcategory', 'task', 'source', 'functions'],
    num_rows: 5208
})